https://scikit-learn.org/stable/modules/clustering.html#dbscan

Baserat på en första titt:
* **DBScan**
* Optics
* Gaussian Mixture

Inputs that are used: <br/>
**eps** = Negihbourhood size will do in intervalls of 50 from 100 to 500<br/>
**min_sample** = minimum number of points in each clusters. This will kept as 1. This ensures that the points that are too far away from other points is not forced into a seperate cluster.

In [ ]:
import numpy as np
import geopandas as gpd
from sklearn.cluster import DBSCAN
import pandas as pd

In [ ]:
gdf = gpd.read_file(r"C:\PhD\Papers\2. MAUP\Namibia\100m\100m.shp")

In [ ]:
pt = gdf.to_crs({ 'init': 'EPSG:3395'})
pt["X"] = pt["geometry"].x
pt["Y"] = pt["geometry"].y

In [ ]:
pt = pt[['X','Y',"id","Pop"]]

In [ ]:
numpis=pt.to_numpy()

In [ ]:
df = pd.DataFrame(numpis)

x = [100, 150, 200, 250, 300, 350, 400, 450, 500]
for val in x: 
    db = DBSCAN(eps=val, min_samples=1).fit(df)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    df["clusters"] = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)

    df.to_csv("test" + str(val) + ".csv")